# Retrieval Augmented LLM App
<img src="../assets/module_5/rap_banner.jpeg">

We have covered quite some ground in terms of understanding and building components for:
- Text Representation
- NLP Tasks
- Pretrained Models and Transfer Learning
- Model Fine-Tuning PEFT
- SFT and LLM Landscape
- Vector Databases
- Libraries and Frameworks

Now we will work through development of an app to showcase how we can leverage all the concepts into a fully functioning system

__Note__: In order to keep things simple, we will leverage most high-level APIs available but the overall setup should be easily extensible

## Why Retrieval Augmentation

While theoretically LLMs are capable of having super long context windows, in real world settings this is a challenge because of:
- Inability/Limitation to ensure LLM focusses on correct sub-sections of the context
- High Memory requirements
- High API Cost
- High Latency , etc.


In order to overcome such challenges, we leverage vector databases to act as intelligent retrieval systems (again powered by LLMs) to:
- Provide focussed context
- Reduce memory, cost and latency requirements
- Unlock super-abilities to use upto-date information
- Offload trivial tasks to expert systems

## Streamlit Enters the Arena

[Streamlit](https://streamlit.io/) is an open-source Python library that makes it easy to create and share beautiful, custom web apps for machine learning and data science.

<a target="_blank" href="https://colab.research.google.com/github/raghavbali/llm_workshop_dhs23/blob/main/module_05/2.retrieval_augmented_llm_app.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Let us Begin with Installation

In [1]:
%%capture
# install dependencies
!pip install -q chromadb
!pip install retry
!pip install -q streamlit
!npm install localtunnel

In [28]:
%%writefile app.py
## import required components

from utils import (
    get_lines,
    load_data,
    get_relevant_documents,
    get_answer,
    create_db,
    sidebar
)
import streamlit as st
chroma_client, db = create_db()

## Setup Page Header and Sidebar
st.set_page_config(page_title="PersonalGPT", page_icon="📖", layout="wide")
st.header("📖PersonalGPT")
sidebar()

## Add Uploader Component
uploaded_file = st.file_uploader(
    "Upload a txt file",
    type=["txt"],
    help="Text files with each sentence acting as a document",
)

## Check if upload is complete
if not uploaded_file:
    st.stop()

## Read uploaded file
try:
    file_data = get_lines(uploaded_file)
    ## Verbose Status update
    st.markdown(f"> Uploaded file has {len(file_data)} lines of text")
except Exception as e:
    st.markdown(f"Could not upload/read file={e}")

## Index Uploaded text file
with st.spinner("Indexing document... This may take a while⏳"):
    load_data(db, documents=file_data)

## status update
st.markdown(f"> Database indexed {db.count()} documents")

## Get User Input
with st.form(key="qa_form"):
    query = st.text_area("Enter Your Query")
    submit = st.form_submit_button("Submit")

## Provide additional Options for citing source
with st.expander("Advanced Options"):
    show_source = st.checkbox("Show Source")

## Generate Output upon button click
if submit:
  # Get relevant documents from DB
  context = get_relevant_documents(query, db)

  # get answer from LLM
  answer,score,error = get_answer(query,context)

  # Showcase response on screen
  st.markdown("#### Answer")
  st.markdown(answer)
  st.markdown(f"> relevance score:{score}")
  st.markdown("---")

  # Add more details if advanced option is chosen
  if show_source:
    st.markdown("> Source(s):")
    st.markdown(f"<i>{context[:100]}...</i>", unsafe_allow_html=True)

Overwriting app.py


In [29]:
!streamlit run app.py &>/content/logs.txt &

In [30]:
!npx localtunnel --port 8501

npx: installed 22 in 2.41s
your url is: https://icy-heads-enjoy.loca.lt
^C
